In [1]:
# IMport dependencies
import pandas as pd
from pandas import set_option
import numpy
import os
import csv

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# import preprocessing from sklearn
from sklearn import preprocessing

# DictVectorizer
from sklearn.feature_extraction import DictVectorizer


def generateX(ohe = True, target = "BTU"):    

    dataFilePath = "dataforfinalproject"
    filename = "RECS_COMBINED_DATA.csv"
    cols_file = "Final_Columns_withCat.csv"

    # read dataset wih all years combined data
    df_recs = pd.read_csv(os.path.join(dataFilePath, filename), low_memory= False)


    # read the columns from Columns csv
    df_cols = pd.read_csv(os.path.join(dataFilePath, cols_file))
#     df_cols.columns

    # Whittle down the dataset to contain only Features required for modeling - X 
    modelDF = df_recs[df_cols[df_cols.FEATURES_MODEL == "Y"].COLUMN_NAME]
    print(f" X Features shape : {modelDF.shape}")

    y_label = df_recs['TOTALBTU']
    print(f"y label shape : {y_label.shape}")

    ### Prepare Data

    # describe the dataframe that will be used for model
    descrDF = modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME].describe()
    
    # transpose to make it easier to obtain columns with values of 99 and <0
    descrDF = descrDF.transpose().reset_index()

    # obtain column names with values 99. 99 indicates missing or unavailable info. this needs to be replaced with MOde
    cols99_2 = descrDF[(descrDF['max'] == 99.0) | (descrDF['min'] < 0) ]['index'].tolist()
    print(f"cols with values as 99 and -2: {cols99_2} \n")

   
    # For all categorical columns, that have 99 and -2 , replace with Columns Mode value#
    # step 1 - Fill na for thse values of 99 and -2
    # Step 2: Fillna with mode

    # step1 
    modelCopy = modelDF.copy()
    modelDF[cols99_2] = modelDF[cols99_2].applymap(lambda r : None if r in [99,-2] else r)


    #step2 :
    modelDF[cols99_2] = modelDF[cols99_2].fillna(modelDF.mode().iloc[0])

    # just for Col EDishw, the values are in -ve  (-9, -8 )so replace it in a separate line
    modelDF['ESDISHW'] = modelDF['ESDISHW'].apply(lambda r : 0 if (r < 0) else r)

    # check if NAN exists
    print(f"Duplicate Count : {modelDF.isnull().values.sum()}")


    modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y") & (df_cols.COLUMN_TYPE == "Categorical")].COLUMN_NAME].describe()

    if(target == "BTU"):
        # Drop Price / Cost related Columns as it is only Consumption we are interested in 
        cost_cols = df_cols[(df_cols['COLUMN_NAME'].str.find("DOL") != -1) & (df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME.tolist()
        modelDF.drop(cost_cols, axis = 1, inplace = True)
        print(modelDF.shape)
    
#     # assign target or output to y
#     y = modelDF_BTU['TOTALBTU']
#     print(f"shape of y is {y.shape}")

    # and drop TOTAL BTU from X set
    X = modelDF.drop(['TOTALBTU'], axis = 1)
    print(f"shape of X is {X.shape}")

    if(ohe):
        ### Apply dict vectorizer 
        # convert the X array into a dict
        X_dict = X.to_dict(orient = "records")
       

        # instantiate a Dictvectorizer object for X
        dv_X = DictVectorizer(sparse=False)   # sparse = False makes the output is not a sparse matrix

        # apply dv_X on X_dict
        X_encoded = dv_X.fit_transform(X_dict)
        
        # return X_encoded
        return X_encoded
    else:
        return X
    

In [3]:
#generateX(ohe = False)

 X Features shape : (22591, 87)
y label shape : (22591,)
cols with values as 99 and -2: ['CELLAR', 'STORIES', 'PRKGPLC1', 'BEDROOMS', 'FUELPOOL', 'SIZRFRI1', 'TYPERFR1', 'AGERFRI1', 'SIZRFRI2', 'TYPERFR2', 'AGERFRI2', 'NUMFREEZ', 'SIZFREEZ', 'AGEFRZR', 'STOVENFUEL', 'OVENUSE', 'AMTMICRO', 'DWASHUSE', 'WASHLOAD', 'WASHTEMP', 'EQUIPM', 'FUELHEAT', 'EQUIPAGE', 'PROTHERM', 'TEMPHOME', 'TEMPGONE', 'TEMPNITE', 'AGECENAC', 'USECENAC', 'NUMBERAC', 'WWACAGE', 'USEWWAC', 'FUELH2O', 'WHEATAGE', 'WHEATSIZ', 'LGTOUTNUM', 'ESDISHW', 'PGASHEAT', 'WOODAMT'] 

Duplicate Count : 0
(22591, 80)
shape of X is (22591, 79)


REGIONC  DIVISION  CELLAR  STORIES  PRKGPLC1  YEARMADE  OCCUPYY  \
0            1         1     1.0     31.0       1.0         9        9   
1            3         7     0.0     10.0       1.0         7        9   
2            4         9     0.0     20.0       1.0         4        1   
3            1         1     1.0     31.0       1.0         1        2   
4            4         9     0.0     10.0       1.0         3        9   
5            4         9     0.0     10.0       0.0         4        9   
6            4         9     0.0     10.0       0.0         3        9   
7            4         9     0.0     20.0       0.0         4        9   
8            3         7     0.0     10.0       0.0         2        9   
9            3         7     0.0     10.0       0.0        10        9   
10           4         9     0.0     10.0       0.0         3        9   
11           3         7     0.0     10.0       0.0         1        3   
12           3         5     0.0     10.0       0.0         2        9   
13           1         2     9.0     10.0       9.0         4        9   
14           1         2     0.0     20.0       1.0         2        9   
15           1         2     9.0     10.0       9.0         1        9   
16           3         5     0.0     10.0       9.0         7        9   
17           1         2     1.0     20.0       0.0         3        9   
18           3         5     9.0     10.0       0.0        10        9   
19           3         5     9.0     10.0       9.0         7        9   
20           3         5     9.0     10.0       9.0         7        9   
21           3         5     9.0     10.0       9.0         7        9   
22           1         2     1.0     31.0       9.0         7        9   
23           1         2     1.0     20.0       0.0         2        9   
24           1         2     9.0     10.0       9.0         2        9   
25           1         2     9.0     10.0       9.0         3        9   
26           1         2     9.0     10.0       9.0         5        2   
27           1         2     1.0     20.0       0.0         1        9   
28           1         2     1.0     20.0       1.0         3        9   
29           3         5     0.0     20.0       0.0         7        2   
...        ...       ...     ...      ...       ...       ...      ...   
22561        4        10     0.0     10.0       1.0         2        3   
22562        4        10     0.0     10.0       0.0         1        6   
22563        3         7     0.0     10.0       1.0         3        5   
22564        4        10     0.0     10.0       1.0         5        7   
22565        4         8     1.0     10.0       1.0         1        8   
22566        2         3     0.0     20.0       1.0         7        7   
22567        3         6     0.0     10.0       1.0         4        8   
22568        2         3     0.0     20.0       0.0         1        5   
22569        2         3     0.0     20.0       1.0         5        5   
22570        1         2     1.0     20.0       1.0         3        7   
22571        3         5     0.0     10.0       0.0         5        6   
22572        1         1     0.0     10.0       1.0         2        3   
22573        2         3     0.0     10.0       1.0         1        8   
22574        1         2     0.0     20.0       0.0         2        2   
22575        1         2     1.0     10.0       1.0         3        8   
22576        1         2     1.0     10.0       0.0         2        7   
22577        1         2     0.0     10.0       1.0         4        8   
22578        1         2     1.0     20.0       1.0         3        6   
22579        4         8     1.0     10.0       0.0         1        7   
22580        3         5     0.0     10.0       0.0         2        3   
22581        3         5     0.0     10.0       1.0         5        6   
22582        1         1     0.0     10.0       0.0         1        4   
22583      